## Step 0: Mounting Google Drive and Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/multimodal-xray-agent
!ls

In [2]:
import torch
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login

In [3]:
login()

## Step 1: Verifying GPU and Environment

In [4]:
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    device = torch.device("cuda")
    print(f"GPU detected: {device_name}")
else:
    device = torch.device("cpu")
    print("GPU not detected. Falling back to CPU.")

print(f"Running on device: {device}")

GPU detected: Tesla T4
Running on device: cuda


## Step 2: Load the Fine-Tuned Model



In [5]:
base_model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [6]:
adapter_path = "./models/llama_lora_adapter"

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [8]:
# Load the LoRA adapter onto the base model
model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
model

In [ ]:
# Load the Generation Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Set to Eval Mode
model.eval()
torch.set_grad_enabled(False)

## Step 3: Evaluation

In [13]:
# We will simulate the context that your agents would provide.
vision_caption_context = "The lungs are hyperexpanded. There are no signs of focal consolidation, pleural effusion, or pneumothorax."
similar_case_context = "Findings are suggestive of chronic obstructive pulmonary disease (COPD)."
user_query_context = "Summarize the key thoracic findings."

In [14]:
# Construct the rich user prompt
user_prompt_content = f"""
A chest X-ray has been provided. Synthesize the following information to generate a concise, professional diagnostic impression.

### AI-Generated Image Caption:
{vision_caption_context}

### Impression from a Similar Case:
{similar_case_context}

### Original User Query:
{user_query_context}

Based on all the provided context, generate the final diagnostic impression:
"""

In [15]:
# Apply Template and Generate ---
messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": user_prompt_content}
]

In [16]:
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [17]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [18]:
output_ids = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id
)

In [19]:
generated_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

In [20]:
# Analyze the Result
print("--- FULL PROMPT SENT TO MODEL ---")
print(prompt)
print("\n" + "="*50 + "\n")
print("--- MODEL'S GENERATED OUTPUT ---")
print(generated_text.split("<|end_header_id|>")[-1].strip())

--- FULL PROMPT SENT TO MODEL ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Jun 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

A chest X-ray has been provided. Synthesize the following information to generate a concise, professional diagnostic impression.

### AI-Generated Image Caption:
The lungs are hyperexpanded. There are no signs of focal consolidation, pleural effusion, or pneumothorax.

### Impression from a Similar Case:
Findings are suggestive of chronic obstructive pulmonary disease (COPD).

### Original User Query:
Summarize the key thoracic findings.

Based on all the provided context, generate the final diagnostic impression:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




--- MODEL'S GENERATED OUTPUT ---
system

Cutting Knowledge Date: December 2023
Today Date: 15 Jun 2025

user

A chest X-ray has been provided. Synthesize the following information to generate a concise, professi

## Step 4: Comparing with the Base Model

In [ ]:
base_model_1 = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
base_model_1.eval()

In [ ]:
torch.set_grad_enabled(False)

In [25]:
output_ids = base_model_1.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id
)

In [26]:
generated_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

In [27]:
# Analyze the Result
print("--- FULL PROMPT SENT TO MODEL ---")
print(prompt)
print("\n" + "="*50 + "\n")
print("--- MODEL'S GENERATED OUTPUT ---")
print(generated_text.split("<|end_header_id|>")[-1].strip())

--- FULL PROMPT SENT TO MODEL ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Jun 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

A chest X-ray has been provided. Synthesize the following information to generate a concise, professional diagnostic impression.

### AI-Generated Image Caption:
The lungs are hyperexpanded. There are no signs of focal consolidation, pleural effusion, or pneumothorax.

### Impression from a Similar Case:
Findings are suggestive of chronic obstructive pulmonary disease (COPD).

### Original User Query:
Summarize the key thoracic findings.

Based on all the provided context, generate the final diagnostic impression:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




--- MODEL'S GENERATED OUTPUT ---
system

Cutting Knowledge Date: December 2023
Today Date: 15 Jun 2025

user

A chest X-ray has been provided. Synthesize the following information to generate a concise, professi